# Zajęcia z programowania 33

## Lista umiejętności

- przeszukiwanie listy w poszukiwaniu konkretnego elementu
- znajdywanie elementu minimalnego
- znajdywanie elementu maksymalnego
- znajdywanie powtórzeń elementów
- znajdywanie par o danej charakterystyce
- znajdywanie NWD, NWW

### Do nauczenia

- transformacja danych
- podróż po grafach
- symulacje warunkowe

## Zadanie

Żółw porusza się po trasie wyznaczonej przez boki sześciokąta. Pierwszy bok przemierza z prędkością
1, drugi i każdy parzysty z prędkością o 3 większą od poprzedniej wartości, trzeci i każdy kolejny
nieparzysty z prędkością o 1 mniejszą od poprzedniej. Po każdych dwóch okrążeniach żółw jest
zmęczony i zamiast o 1 zmniejsza swą prędkość o 10.

Napisz jednoparametrową funkcję `ile(n)`, której wynikiem będzie prędkość żółwia na n-tym boku.
Parametr n może przyjmować wartości od 1 do 10 000.

### Przykłady
- Wynikiem `ile(4)` jest `6`.
- Wynikiem `ile(14)` jest `7`.
- Wynikiem `ile(100)` jest `30`.